###**Modelo con preprocesado C y RandomForestClassifier**

####**Cargar librerías**

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
from sklearn.model_selection import cross_val_score, KFold

####**Carga del conjunto de datos Train y Test**

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."  #Se establece la variable de entorno al directorio actual

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia  # Se descargan los archivos de la competencia directamente desde la API de Kaggle

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip # Se descomprime el fichero para acceder al contenido

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train = pd.read_csv('train.csv')  # Carga del conjunto de datos de entrenamiento
test = pd.read_csv('test.csv')  # Carga del conjunto de datos de test

##**Limpieza de datos**


#####**Eliminación de columnas**

In [ ]:
del(train["ID"]) # No es relevante para el modelo, ya que no tiene mucha relación con el rendimiento ni aporta información predicitva
del(train["F_TIENEINTERNET.1"]) # Columna duplicada de 'F_TIENEINTERNET.1', se elimina para evitar redundancia
del(train["E_PRIVADO_LIBERTAD"]) # Tiene una proporción mínima de casos en una categoría, por lo tanto no es relevante
del(train["PERIODO_ACADEMICO"]) # Se elimina debido al desbalance de datos entre periodos.

In [ ]:
del(test["F_TIENEINTERNET.1"])
del(test["E_PRIVADO_LIBERTAD"])
del(test["PERIODO_ACADEMICO"])

#####**Verificación de las columnas restantes**

In [ ]:
print(train.columns) # Estas son las variables que permanecen después de la limpieza del dataset
# Ordenar las columnas para que el RENDIMIENTO_GLOBAL quede de ultima y pueda ser consistente con los datos de test
otras = [c for c in train.columns if c != "RENDIMIENTO_GLOBAL"]
# Reordenamos
train = train[otras + ["RENDIMIENTO_GLOBAL"]]
print(train.columns)

Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO',
       'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')
Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO',
       'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


####**Tratamiento de Columnas con Datos Faltantes**



In [ ]:
train = train.fillna("missing") # Se reemplazan los valores faltantes de las variables categóricas por categoría "missing"

In [ ]:
test = test.fillna("missing") # Se reemplazan los valores faltantes de las variables categóricas por categoría "missing"

##**Codificación de Variables Categóricas con One Hot**

In [ ]:
#Estas funciones permiten trasnformar las variables categóricas en formato One-Hot
def to_onehot(x): # Convierte una variable categórica a formato One Hot
    values = np.unique(x)
    r = np.r_[[np.argwhere(i == values)[0][0] for i in x]]
    onehot_sparse = sparse.csr_matrix(
        (np.ones(len(r)), (np.arange(len(r)), r)),
        shape=(len(r), len(values))
    )
    return onehot_sparse, values

# Reemplaza una columna por su versión One Hot
def replace_column_with_onehot(d, col):
    assert sum(d[col].isna()) == 0, "column must have no NaN values"
    k_sparse, values = to_onehot(d[col].values)
    k = pd.DataFrame.sparse.from_spmatrix(
        k_sparse,
        columns=["%s_%s" % (col, values[i]) for i in range(k_sparse.shape[1])],
        index=d.index
    )
    r = k.join(d)
    del (r[col])
    return r

# Aplica One-Hot Encoding a múltiples columnas categóricas
def aplicar_onehot_a_varias(df, columnas):
    df_datos = df.copy()
    for col in columnas:
        df_datos = replace_column_with_onehot(df_datos, col)
    return df_datos


In [ ]:
  # Aplica One Hot a las variables categóricas seleccionadas en df_train
df_train = aplicar_onehot_a_varias(
    train,
    [
        "F_TIENEINTERNET",
        "F_TIENELAVADORA",
        "F_TIENEAUTOMOVIL",
        "E_PAGOMATRICULAPROPIO",
        "F_TIENECOMPUTADOR",
        "E_PRGM_DEPARTAMENTO",
        "E_PRGM_ACADEMICO"
    ]
)

In [ ]:
# Aplica One Hot a las variables categóricas seleccionadas en df_test
df_test = aplicar_onehot_a_varias(
    test,
    [
        "F_TIENEINTERNET",
        "F_TIENELAVADORA",
        "F_TIENEAUTOMOVIL",
        "E_PAGOMATRICULAPROPIO",
        "F_TIENECOMPUTADOR",
        "E_PRGM_DEPARTAMENTO",
        "E_PRGM_ACADEMICO"
    ]
)

##**Codificación de variables categóricas ordinales**

In [ ]:
def codificar_ordinal(df, mapeos):
    # Aplica codificación ordinal a las columnas especificadas en 'mapeos'
    # cada categoría es reemplazada por un número según su jerarquía.
    df_copia = df.copy()
    for col, mapa in mapeos.items():
        if col in df_copia.columns:
            df_copia[col] = df_copia[col].map(mapa)

    return df_copia

In [ ]:
mapeos_train = {
    'E_VALORMATRICULAUNIVERSIDAD': {
        'missing': -1,
        'No pagó matrícula': 0,
        'Menos de 500 mil': 0.25,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.25,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Más de 7 millones': 8
    },
    'E_HORASSEMANATRABAJA': {
        'missing': -1,
        '0': 0,
        'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4
    },
    'F_ESTRATOVIVIENDA': {
        'missing': -1,
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    },
    'F_EDUCACIONPADRE': {
        'missing': -2,
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10,
    },
    'F_EDUCACIONMADRE': {
        'missing': -2,
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10,
    },
    'RENDIMIENTO_GLOBAL': {
        'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3
    }
}


In [ ]:
mapeos_test = {
    'E_VALORMATRICULAUNIVERSIDAD': {
        'missing': -1,
        'No pagó matrícula': 0,
        'Menos de 500 mil': 0.25,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.25,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Más de 7 millones': 8
    },
    'E_HORASSEMANATRABAJA': {
        'missing': -1,
        '0': 0,
        'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4
    },
    'F_ESTRATOVIVIENDA': {
        'missing': -1,
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    },
    'F_EDUCACIONPADRE': {
        'missing': -2,
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10,
    },
    'F_EDUCACIONMADRE': {
        'missing': -2,
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10,
    }
}


In [ ]:
#Se aplica la codificación ordinal a las variables categóricas en df_train según los mapeos definidos
df_train_map = codificar_ordinal(df_train, mapeos_train)

In [ ]:
#Se aplica la codificación ordinal a las variables categóricas en df_test según los mapeos definidos
df_test_map = codificar_ordinal(df_test, mapeos_test)

##**Normalización de Columnas**

In [ ]:
# Se crea el objeto escalador MinMax y se aplica las columnas seleccionadas
escalar = MinMaxScaler()
# Columnas a normalizar
columnas_a_normalizar = [
    "E_VALORMATRICULAUNIVERSIDAD",
    "INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]
# Ajustar el escalador a los datos y transformar las columnas
df_train_map[columnas_a_normalizar] = escalar.fit_transform(df_train_map[columnas_a_normalizar])
df_test_map[columnas_a_normalizar] = escalar.fit_transform(df_test_map[columnas_a_normalizar])

In [ ]:
df_train_final = df_train_map.copy()
df_test_final = df_test_map.copy()

###**Modelo RandomForestClassifier**



Decidimos utilizar Random Forest, uno de los algoritmos de aprendizaje supervisado enseñado en la teoría, el cual combina múltiples árboles de decisión para aumentar la precisión de las predicciones. De manera preliminar, calculamos el accuracy usando la librería accuracy_score para evaluar cómo se comporta el modelo.

In [ ]:
# Variable objetivo
y = df_train_final["RENDIMIENTO_GLOBAL"]

# Predictoras
X = df_train_final.drop(columns=["RENDIMIENTO_GLOBAL"])

# Entrenamiento
rf = RandomForestClassifier(
    n_estimators=10,
    max_depth=5
)

rf.fit(X, y)

# Predicciones
y_pred = rf.predict(X)

# Resultados
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)
print("Dimensiones X:", X.shape, "Dimensiones y:", y.shape)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Accuracy: 0.3691552346570397
Dimensiones X: (692500, 1003) Dimensiones y: (692500,)


###**Revisión del Accuracy con Datos de Train**

Se realiza una partición de los datos de train en 70% entrenamiento y 30% prueba para evaluar el desempeño del modelo.

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.3
val_size  = test_size / (1 - test_size)

print("Dimensiones:", X.shape, y.shape)
print("test size =", test_size)
print("val size  =", val_size)

Xtv, Xts, ytv, yts = train_test_split(
    X, y,
    test_size=test_size,
    stratify=y,
    random_state=7
)

print("Train:", Xtv.shape, "Test:", Xts.shape)

#Accuracy
y_pred = rf.predict(Xts)
print("Accuracy en test:", accuracy_score(yts, y_pred))

Dimensiones: (692500, 1003) (692500,)
test size = 0.3
val size  = 0.4285714285714286
Train: (484750, 1003) Test: (207750, 1003)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Accuracy en test: 0.3689578820697954


En esta prueba se obtiene un 36.8% de accuracy

####**Técnicas de Validación Cruzada**

#####**Uso de ShuffleSplit**

Se utilizó validación cruzada con ShuffleSplit. Se realizaron tres particiones del dataset, donde en cada split se tomó un 70 % de los datos para entrenamiento y un 30 % para test. El modelo se entrenó con los hiperparámetros establecidos en el conjunto de entrenamiento y se calculó la métrica accuracy.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, ShuffleSplit
import numpy as np

# Modelo
estimator = RandomForestClassifier(n_estimators=10, max_depth=5)

# ShuffleSplit con 3 splits
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=7)

scores = cross_val_score(estimator, X, y, cv=cv, scoring='accuracy')

print("CV scores:", scores)
print("Mean ± std:", np.mean(scores), "+-", np.std(scores))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklea

CV scores: [0.36070758 0.36367268 0.37115764]
Mean ± std: 0.36517930204572807 +- 0.004397223664362474


Los resultados obtenidos muestran un accuracy promedio de 36.51 %, lo que indica que el modelo acierta aproximadamente un tercio de las veces sobre los datos de prueba. La desviación estándar de 0.0043 demuestra que el rendimiento es consistente y que el modelo tiende a mantener un desempeño cercano a la media.

#####**Uso de KFold**

Para reforzar la evaluación del desempeño del modelo se aplicó validación cruzada utilizando KFold. Se realizaron tres particiones (splits) del dataset, donde en cada fold se entrenó el modelo y se calculó la métrica accuracy.

In [ ]:
# Modelo
estimator = RandomForestClassifier(n_estimators=10, max_depth=5)

# KFold con 3 splits
cv = KFold(n_splits=3, shuffle=True, random_state=7)

# Puntajes
scores = cross_val_score(estimator, X, y, cv=cv, scoring='accuracy')
print("CV scores:", scores)
print("Mean ± std:", np.mean(scores), "+-", np.std(scores))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklea

CV scores: [0.36833395 0.37403664 0.36891606]
Mean ± std: 0.3704288838915952 +- 0.002562114400841229


Los resultados muestran un accuracy promedio de 37.04%, con una desviación estándar muy baja (0.0025), lo cual es coherente con la validación anterior.

####**Generación del CSV - Kaggle**

In [ ]:
# Alinear columnas de test con las de train
# Se eliminan porque los datos de train no tienen información sobre esos programas académicos
X_test_aligned = df_test_final.reindex(columns=X.columns, fill_value=0)

# Generar predicciones
y_pred_num = rf.predict(X_test_aligned)

# Mapear números nuevamente al original
num_to_label = {0:'bajo', 1:'medio-bajo', 2:'medio-alto', 3:'alto'}
y_pred_labels = [num_to_label[i] for i in y_pred_num]

# Guardar en CSV usando la columna ID de test
resultado = pd.DataFrame({
    "ID": df_test_final["ID"],  # columna identificadora
    "RENDIMIENTO_GLOBAL": y_pred_labels
})

resultado.to_csv("predicciones_test.csv", index=False)
print("Predicciones generadas")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Predicciones generadas
